# Notebook for PURVIEW schema analysis
## Requirements
1. A **Purview** environment with a dedicated pool database
2. A **Synapse** workspace to run that spark notebook (smallest cluster is enough)
3. A **service principal** with permission in purview

## Configuration
1. Populate all the **paramenters** in the first cell
2. **Run** all the notebook cells

## Results
A list of tables with the ammount of columns to verify the absence of schema in some tables. 

In [ ]:
# Parameters 
workspacename = "myworkspace"
databasename = "mydb"
purviewname = "mypurview"
storageaccount = "mystorageaccount"
container = "mycontainer"
keyvaultname = "mykeyvault"
secretname = "mysecret"

csvroot = 'abfss://%s@%s.dfs.core.windows.net/%s/' % (storageaccount, container,"schema_analysis")

#This variable has to be one of both engines -serverless- or -dedicated-
enginetype = "serverless"

tenant_id = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
client_id = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"
#We should store secrets in a keyvault where only the Workspace MI has read permissions 
client_secret = TokenLibrary.getSecret(keyvaultname, secretname)
debug = False

In [ ]:
import os 
import requests 
import json
import pandas as pd 
from notebookutils import mssparkutils 
from pyspark.sql import * 
from pyspark.sql.functions import * 
from pyspark.sql.types import *
import jmespath

In [ ]:
def azuread_auth(tenant_id: str, client_id: str, client_secret: str, resource_url: str):
    url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
    payload= f'grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}&resource={resource_url}'
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
        }
    response = requests.request("POST", url, headers=headers, data=payload)
    access_token = json.loads(response.text)['access_token']
    return access_token
	
	
def get_all_assets(path: str, data_catalog_name: str, azuread_access_token: str, max_depth=1):
    url = f"https://{purviewname}.catalog.purview.azure.com/api/browse?api-version=2021-05-01-preview"
    headers = {
            'Authorization': f'Bearer {token}',
            'Content-Type': 'application/json'
            }
    payload="""{
                "limit": 100,
                "offset": null,
                "path": "%s"
                }""" % (path)         
    response = requests.request("POST", url, headers=headers, data=payload)
    #print(response.text)
    li = json.loads(response.text)
    # Return all files
    for x in jmespath.search("value", li):
        if jmespath.search("isLeaf", x):
            yield x
    if max_depth > 1:
        for x in jmespath.search("value", li):
            if jmespath.search("isLeaf", x):
                continue
            for y in get_all_assets(jmespath.search("path", x), data_catalog_name, azuread_access_token, max_depth - 1):
                yield y
    else:
        for x in jmespath.search("value", li):
            if jmespath.search("!isLeaf", x):
                yield x

def getcolumns(typeName: str, assetfqn: str, azuread_access_token: str): 
    prefix = f"https://{purviewname}.catalog.purview.azure.com/api/atlas/v2/entity/uniqueAttribute/type"
    url = f"{prefix}/{typeName}?attr:qualifiedName={assetfqn}" 
    headers = { 
            'Authorization': f'Bearer {azuread_access_token}', 
            'Content-Type': 'application/json' 
            } 
    response = requests.request("GET", url, headers=headers) 
    if debug: 
        print(response.text)
    result = json.loads(response.text)['referredEntities'].values() 
    #dbschema = json.loads(response.text)['entity']['relationshipAttributes']['dbSchema']['displayText'].values()[0]
    mycolumns = [] 
    for column in result: 
        item = {} 
        item['typename'] = column['typeName']
        #item['dbschema'] = dbschema
        item['name'] = column['attributes']['name'] 
        item['guid'] = column['guid'] 
        mycolumns.append(item) 
    mycolumns = pd.DataFrame.from_dict(mycolumns) 
    return mycolumns

In [ ]:
token = azuread_auth (tenant_id, client_id, client_secret, "https://purview.azure.net")
# token = azuread_auth(tenantid,clientid,secret,"https://purview.azure.net")

# if schema predefined path = f"/azure_synapse_workspace#syn-mfceus2aaihubtst11/azure_synapse_dedicated_sql_db#{databasename}/azure_synapse_dedicated_sql_schema#{schemaname}"
purviewpath = f"/azure_synapse_workspace#{workspacename}/azure_synapse_{enginetype}_sql_db#{databasename}"

items = get_all_assets(purviewpath,purviewname,token,max_depth=2)
mydict = []
for item in items:
    mydict.append(item)
if len(mydict)>0:
    tables=spark.createDataFrame(mydict)
else:
    print("THE TABLE DATA is empty, please check that this database has tables in Purview")
if debug:
    display(tables)

In [ ]:

# baseurl="https://"+mssparkutils.env.getWorkspaceName()+".dev.azuresynapse.net"
# headers = {"Authorization": "Bearer "+mssparkutils.credentials.getToken("Synapse")}
# constructed_url = baseurl + "/databases/"+lakedatabase+"/tables?api-version=2021-04-01"
# response = requests.get(constructed_url, headers=headers)
# response = json.loads(response.text)
# print(response)
items = 0
# tables = spark.sql('show tables in ' + lakedatabase)
print("          Table       - Count  --  fqn on mssql server / purview ")
output = []
for item in tables.collect():
    outitem = {}
    notfinish = True
    tablename=item['name']
    fqn = item['qualifiedName']
    typename = item['entityType']
    assetfqn = item['qualifiedName']
    #f"mssql://datamasked-ondemand.sql.azuresynapse.net/{lakedatabase}/dbo/{tablename}"
    token = azuread_auth(tenant_id,client_id,client_secret,"https://purview.azure.net")
    # result = set_desc(typename, assetfqn,description,token)
    try:
        dfcolumns = getcolumns(typename, assetfqn,token)
        columncount = dfcolumns.shape[0]
        if columncount == 0:
            outitem['tablename'] = tablename
            outitem['columncount'] = columncount
            outitem['fqn'] = fqn
            output.append(outitem)
            items += 1
        print("%20s" % (tablename) + f"  -   {columncount}    -- {fqn}")
    except Exception as e: 
        print(e)
pd.DataFrame(output).to_csv(csvroot + databasename +"_emptytables.csv",storage_options = {'linked_service' : 'datamasked-WorkspaceDefaultStorage'})
print(f"\n Items without columns: {items}")